# Contents <a id='top'></a>

1. <a href=#eda>Data Exploration</a>
1. <a href=#intro>Introduction to Regression</a>
1. <a href=#resid>Residual Analysis</a>
1. <a href=#simple>A Simple Model</a>
1. <a href=#improvement>An Improvement</a>
1. <a href=#ref>References and Links</a>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

<a id='eda'></a>
# 1. Exploring Taiwan Dataset
<a href=#top>(back to top)</a>

For this tutorial, we shall work with a data set from the UCI machine learning repository. It contains real estate prices in the Xindian district of Taiwan. Our goal is to answer the following question:

> How well can we explain real-estate prices in Taiwan?

Let us first explore the dataset with Python. Make a few plots, and share what you see.

In [ ]:
re2 = pd.read_csv("../data/taiwan_dataset.csv")
re2.loc[:, 'trans_date':].describe().round(3)

In [ ]:
pd.plotting.scatter_matrix(re2.loc[:, ['house_age', 'dist_MRT', 'num_stores', 'Xs', 'Ys', 'price']], figsize=(15, 15));

<a id='intro'></a>
# 2. Introduction to Regression
<a href=#top>(back to top)</a>

## Model Definition

A linear regression model relates a dependent variable ($y$) to several independent variables through an equation of the form:

$$ y_i = \beta_0 + \beta_1 x_{1,i} + \beta_2 x_{2,i} + \cdots + \beta_{p-1} x_{p-1,i} + \epsilon_i $$

The $y$ and $x$ are observed, but the $\beta$'s and the $\epsilon$'s are not. The index $i$ refers to a particular observation. Let us assume we have $n$ observations.

## Parameter Estimation

To estimate the $\beta$ values, we minimise a cost function. One of the most commonly used ones is the least squares objective function:

$$
\sum_{i=1}^n (y_i - \beta_0 - \beta_1 x_{1,i} - \beta_2 x_{2,i} - \ldots \beta_{p-1} x_{p-1,i})^2
$$

We denote the optimal estimates as $\hat{\beta}_0, \hat{\beta}_1, \ldots, \hat{\beta}_{p-1}$. 

## Residuals

Once we have the $\hat{\beta}$'s, we can compute **fitted values** by applying the equation back to the observed $x$ values. We can then compute **residuals**. There will be $n$ residuals, one for each observation that we have. We are going to use the residuals to help improve our model fit.

$$
\hat{y_i} =  \hat{\beta}_0 + \hat{\beta}_1 x_{1,i} + \hat{\beta}_2 x_{2,i} +
\cdots + \hat{\beta}_{p-1} x_{p-1,i}
$$ 

Residuals are defined to be 
$$
r_i = y_i - \hat{y_i}
$$

Here is what me mean, in a **simple** linear regression setting. The red values are the fitted values.

<img src="../figs/residuals-1.png" style="width: 600px;"/>

<a id='resid'></a>
# 3. Residual Analysis
<a href=#top>(back to top)</a>

## Influential Points

Some points in our dataset will influence the coefficients, and thus fitted values, and thus the residuals more than others. We need to identify these and study them, because we do not want our model driven by these chance occurences or wild points. Typically, influential points have one or both these properties:

1. They may be far away from their neighbours in the predictor space.
2. They may have an unduly large residual.

### On Coefficients

To see if point $i$ is influential on coefficient $j$, we 

* Estimate the model coefficients with all the data points.
* Leave out the observations $(x_i, y_i)$ one at a time and re-estimate the model coefficients.
* Compare the $\beta_j$'s ($n$ of them) from step 2 with the original estimate from step 1.  

<img src="../figs/inf_coef-1.png" style="width: 450px;"/>

### On Fitted Values

To assess the influence of each point on the fitted values, we use Cook's Distance, which is given by this formula:

$$
D_i = \frac{\sum_{j=1}^n (\hat{y}_j - \hat{y}_{j,-i})^2}{p s^2}$$

where 
*   $\hat{y}_{j,-i}$ is the fitted value for $y_j$ using the model that is fitted without observation $i$.
* $p$ is the number of predictors, including the intercept. In the diagrams above, we are considering only one predictor $x$ for the response variable $y$, and thus call it "simple" as opposed to "multiple" linear regression. Hence $p=2$.
* $s^2$ is a measure of the variance of $\epsilon$; it is computed as
$$ 
  s^2 = \frac{1}{n-p} \sum_{i=1}^n r_i^2
$$

We typically set aside and investigate the 3 - 4 most influential points at each iteration of our analysis.

## Residual Plots

We plot residuals against
* fitted values.
* explanatory variables one at a time.


<img src="../figs/residual_patterns-1.png" style="width: 800px;"/>

## Other Residual Plots

Other plots that could (and should) be made are
* Residuals against time order of data collection.
* Residuals against new variables to be considered for addition into the model.
* Residuals against a product of existing variables to check for interaction.
* QQ plots

<img src="../figs/qq_ex_1-1.png" style="width: 800px;"/>


## Assessing Model Fit

We can compute the $R^2$ of a model to assess how well it explains the variability in the observed values.

$$ 
R^2 = 1 - \frac{\sum_{i=1}^n r^2_i}{\sum_{i=1}^n (y_i - \bar{y})^2}
$$

A value close to 1 is better than a value close to 0. However, the $R^2$ can be misleading since it can be artificially increased by adding more predictors. Hence we usually look at the adjusted $R^2$, which penalises a model for using too many explanatory variables.

$$ \text{adjusted } R^2 = 1 - \frac{\frac{1}{n-1-p}\sum_{i=1}^n
r^2_i}{\frac{1}{n-1}\sum_{i=1}^n (y_i - \bar{y})^2}$$

Another statistic I tend to compute is the Predictive Residual Sum of Squares. It yields some indication of how well the model can generalise.

  $$ \sum_{i=1}^n (\hat{y}_{-i} - y_i)^2 $$
  
To get this quantity back to the scale of the original \(y\)-variable, we compute the root mean squared prediction error: 
$$ 
  \text{RMSPE} = \sqrt{\frac{1}{n}\sum_{i=1}^n (\hat{y}_{-i} - y_i)^2}
$$


<a id='simple'></a>
# 4. A Simple Model
<a href=#top>(back to top)</a>

Let's begin with a simple model:
$$
\text{price} = \beta_0 + \beta_1 \text{age} + \beta_2 \text{dist} + 
\beta_3 \text{(num. of stores)}  + \beta_4 \text{X} + \beta_5 \text{Y} + \epsilon
$$

Note that $p=6$ in this case.

In [ ]:
import statsmodels.api as sm
import statsmodels.stats.outliers_influence as oinf

In [ ]:
X = re2[['house_age', 'dist_MRT', 'num_stores', 'Xs', 'Ys']]
y = re2.price

In [ ]:
X = sm.add_constant(X)

In [ ]:
s_model = sm.OLS(y, X)

In [ ]:
s_results = s_model.fit()

In [ ]:
s_results.summary()

The adjusted $R^2$ is approximately 0.57, but there is a warning about high condition number and numerical instability. If we look at the coefficients, the MRT distance is very small yet highly significant. The scale of the MRT distance is too large compared to the rest. This is probably causing the matrix inversion to be unstable. Additionally, the coefficient for the X-variable does not seem to be significant.

In [ ]:
s_results_inf = s_results.get_influence()
r_star = s_results_inf.get_resid_studentized_external()

In [ ]:
RMSPE = np.sqrt((s_results_inf.resid_press**2).mean())

RMSPE

Here is a panel of residual plots. Take a close look at them, and identify which points you should investigate further.

In [ ]:
f, axs = plt.subplots(2, 3, squeeze=False, figsize=(14,8))

for i,cc in enumerate(X.columns[1:]):
    plt.subplot(2,3, i+1)
    plt.scatter(X[cc], r_star, alpha=0.8)
    plt.title(cc)

plt.subplot(236)
plt.scatter(s_results.fittedvalues, r_star, alpha=0.8)
plt.title('Fitted Values');

Now here are the differences in beta coefficient values, along with the Cook's distance plot. There is one point that is very influential. Isolate this point.

In [ ]:
s_results_inf.dfbetas[:5, :].round(3)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(131)
plt.scatter(np.arange(414), s_results_inf.dfbetas[:,2]);plt.title('betadiff: MRT');plt.xlabel('id');
plt.subplot(132)
plt.scatter(np.arange(414), s_results_inf.dfbetas[:,3]);plt.title('betadiff: num. stores');plt.xlabel('id');
plt.subplot(133)
plt.scatter(np.arange(414), s_results_inf.cooks_distance[0]);plt.title('Cooks Distance');plt.xlabel('id');

In [ ]:
from scripts.inference import check_normality as cn

cn(pd.Series(r_star))
#oinf.variance_inflation_factor(X.iloc[:,1:].to_numpy(), 1)

Here are some of the things we can try to remedy:

1. distance should be transformed.
2. There is still a little remaining curvature in age.
3. There is one outlier that is also influential (on fits and on the coefficients).
4. X is not significant.
5. Is the residuals vs. fitted indicating right-skewness?

<a id='improvement'></a>
# 5. An Improvement
<a href=#top>(back to top)</a>

Let's try two things first, to see if we can improve the adjusted $R^2$: take a log transform of the distance, and drop the influential point (point number 271).

In [ ]:
X2 = re2.loc[:, ['house_age', 'num_stores', 'Xs', 'Ys']]
X2.loc[:, 'ldist'] = np.log(X.dist_MRT)

In [ ]:
y2 = y.drop(270)
X2 = X2.drop(index=270)

X2 = sm.add_constant(X2)

In [ ]:
s2_model = sm.OLS(y2, X2)
s2_results = s2_model.fit()
s2_results.summary()

The warning about numerical instability disappears, the coefficient for distance is significant and the adjusted $R^2$ has gone up. Take some time to do the following, and compare it to the earlier model:

1. Residual plots
2. RMSPE statistic

Are we doing better?

<a id='ref'></a>
# 6. References
<a href=#top>(back to top)</a>

1. [statsmodels API](https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html#examples-notebooks-generated-ols--page-root) The documentation is not as good as sklearn, but still decent.
2. Applied Regression Analysis, by Draper and Smith.